# Bitcoin 30 Days Price changes with CoinGecko API

## Installing the needed dependencies for the API

In [ ]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

## Importing the necessary API libraries 

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

## Creating our API Instance

#### We will be using the CoinGecko API to create one of the candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API, we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

#### Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin we want, vs_currency is the currency we want the price in, and days is how many days back from today we want.

In [7]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [ ]:
type(bitcoin_data)
bitcoin_data

## Data Wrangling

#### The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [ ]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

#### Finally lets turn this data into a Pandas DataFrame.

In [ ]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
data

In [ ]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))
data['date']

#### Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [19]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

date         Price                                          
                         min           max         first          last
0   2021-10-17  59498.588341  61469.747869  61051.988324  59498.588341
1   2021-10-18  60565.345082  62794.042756  60802.774179  61291.182960
2   2021-10-19  61795.839004  64071.376373  61795.839004  64071.376373
3   2021-10-20  63753.896855  67118.891946  64123.519102  66318.127393
4   2021-10-21  62705.137288  66424.674259  65974.642577  62978.457697
5   2021-10-22  60768.884213  63901.747336  62386.667168  60882.921198
6   2021-10-23  60405.151363  61899.451064  60405.151363  61091.801728
7   2021-10-24  60171.631919  61572.140434  61532.719403  61315.382786
8   2021-10-25  60921.603746  63917.434158  60921.603746  62891.775659
9   2021-10-26  61972.932606  63333.083275  63113.777845  62336.146308
10  2021-10-27  58708.953098  61525.057147  60776.030045  58995.435119
11  2021-10-28  58501.391492  61803.521503  59021.085399  61605.245502
12  2021-10-29  60698.166179  62912.085257  61076.700751  62463.092709
13  2021-10-30  61331.287313  62518.608007  62518.608007  61888.968402
14  2021-10-31  60448.852454  62349.611660  61687.021522  61564.920919
15  2021-11-01  59979.030236  62302.431084  61472.844726  60950.819200
16  2021-11-02  60876.487803  64210.946538  61185.320289  63035.316596
17  2021-11-03  61947.974470  63546.464313  62954.452300  63205.480873
18  2021-11-04  61063.504554  63025.089676  62913.679251  61467.790330
19  2021-11-05  60931.950483  62468.877431  61325.802823  61275.146978
20  2021-11-06  60493.366134  61634.282542  61275.780558  61610.838755
21  2021-11-07  61566.480378  63005.039479  61566.480378  63005.039479
22  2021-11-08  63036.736127  66509.587623  63036.736127  66389.783854
23  2021-11-09  66660.734224  68425.843440  66660.734224  67775.335325
24  2021-11-10  65332.537632  68640.160350  67549.367640  65332.537632
25  2021-11-11  64712.056838  65735.078555  65261.569194  65265.112166
26  2021-11-12  63091.010593  65746.739570  65419.411416  64341.158049
27  2021-11-13  63691.403147  64991.705536  64111.227812  64471.931313
28  2021-11-14  63874.891514  65219.483033  64521.648875  64599.942505
29  2021-11-15  63991.944474  66136.912461  64763.012845  64241.511398
30  2021-11-16  61018.843343  64069.393251  64069.393251  61107.227691

## Data Visualization 

#### Finally we are now ready to use plotly to create our Candlestick Chart.

In [18]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()